In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
from skimage import feature 

In [2]:
def discard(image):
    image = np.uint8(image)
    _, im_label, stts, _ = cv2.connectedComponentsWithStats(image, connectivity=4)
    msk = np.isin(im_label, np.where(stts[:, cv2.CC_STAT_WIDTH] > 500)[0])
    image[msk] = 0
    return image

In [14]:
#import image
image = cv2.imread('input.png')

#cv2.imshow('orig',image)
#cv2.waitKey(0)

#grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# cv2.imshow('gray',gray)
# cv2.waitKey(0)

#binary
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
# cv2.imshow('second',thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Discarding long connected components
image_without_lines = discard(thresh.copy())

image_with_just_lines = cv2.bitwise_xor(thresh, image_without_lines)


# image_with_just_lines = cv2.resize(image_with_just_lines, (1280,1024) )

# cv2.imshow("horizontal_line", image_with_just_lines)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
sum_horizontal = np.sum(image_with_just_lines,axis=1)

lines_indexes = np.argwhere(sum_horizontal>2000).flatten()


y_start = lines_indexes[ np.argmax(lines_indexes > (lines_indexes[0]+150) ) ]
y_end = max(lines_indexes)

# thresh = thresh[x_start:x_end,]

# thresh = cv2.resize(thresh, (1280,1024) )

# cv2.imshow("horizontal_line", thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


#dilation
kernel = np.ones((5,100), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=2)

# img_dilation = cv2.resize(img_dilation, (1280,1024) )
# cv2.imshow('dilated',img_dilation)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#find contours
ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
lines_of_text = []
for i, ctr in enumerate(sorted_ctrs):
    # Get bounding boxl
    x, y, w, h = cv2.boundingRect(ctr)
    if(y< y_end and y>y_start and h>80):
        # Getting ROI
        roi = image[y:y+h, x:x+w]
        
        lines_of_text.append((y,y+h,x,x+w))
        
        # show ROI
        cv2.rectangle(image,(x,y),( x + w, y + h ),(90,0,255),2)


# image = cv2.resize(image, (1280,1024) )
# cv2.imshow('marked areas',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

<class 'numpy.ndarray'>


### Wavelet Transform

In [15]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar'):
    imArray = cv2.imread(img)
    imArray = imArray[lines_of_text[0][0]:lines_of_text[0][1],lines_of_text[0][2]:lines_of_text[0][3]  ]
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_BGR2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)
    return imArray_H
    #Display result
#     cv2.imshow('image',imArray_H)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

imArray_H = w2d("input.png",'db4')

### local binary pattern

In [16]:
img_lbp = feature.local_binary_pattern(imArray_H,8,1)

cv2.imshow('image',img_lbp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
print(type(image))

<class 'numpy.ndarray'>


In [23]:
xdim = img_lbp.shape[0]
ydim = img_lbp.shape[1]
xgrid = 1
ygrid = 8
xstep = xdim//xgrid
ystep  = ydim//ygrid
list_histograms = []
img_lbp=np.float32(img_lbp)
for i in range(xgrid):
    xstart = 0
    ystart = 0
    for j in range(ygrid):
        histogram = cv2.calcHist([img_lbp[xstart:xstart+xstep,ystart:ystart+ystep]],[0],None,[256],[0,256])
        list_histograms.append(histogram)
        xstart +=xstep
        ystart +=ystep
        
print(list_histograms)

[array([[1.34e+03],
       [4.63e+02],
       [2.60e+01],
       [6.80e+01],
       [4.01e+02],
       [4.90e+01],
       [7.50e+01],
       [2.85e+02],
       [2.00e+01],
       [1.30e+01],
       [9.00e+00],
       [6.00e+00],
       [7.40e+01],
       [1.80e+01],
       [9.90e+01],
       [3.58e+02],
       [4.44e+02],
       [3.51e+02],
       [1.70e+01],
       [1.50e+01],
       [5.10e+01],
       [2.00e+01],
       [3.60e+01],
       [3.00e+01],
       [6.80e+01],
       [1.70e+01],
       [1.20e+01],
       [1.20e+01],
       [2.59e+02],
       [2.90e+01],
       [3.40e+02],
       [3.97e+02],
       [4.10e+01],
       [1.00e+01],
       [1.00e+00],
       [9.00e+00],
       [1.20e+01],
       [4.00e+00],
       [6.00e+00],
       [2.00e+01],
       [1.00e+00],
       [1.00e+00],
       [2.00e+00],
       [1.00e+00],
       [6.00e+00],
       [0.00e+00],
       [1.00e+01],
       [1.90e+01],
       [5.80e+01],
       [1.30e+01],
       [2.00e+00],
       [3.00e+00],
       [3.7